In [ ]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_DAT import AutoDAT
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ

from lib.file.ROI_reader import ROIFileReader
from lib.analysis.laminar_dist import LaminarROI, Line
from lib.analysis.align import ImageAlign
from lib.utilities import *

####################################################################################
#    Scaffold for Cory's automatic data export
#  Input:
#     ZDA files, each with a corresponding ROI dat file
#     A specified PhotoZ .pre file to be used for all ZDA
#  Output:
#     Half-amp latency, half-width, amp, SNR for every ROI/trial 
####################################################################################

In [ ]:
# Fill all these variables to specify your input directory / format
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's/hw_and_amp_scatter/" # end this in a /
photoZ_preference_file = None
if photoZ_preference_file is None:
    raise Exception("Please provide full path to photoZ preference file in the variable photoZ_preference_file")

n_trials_per_recording = 5
roi_file_keyword = 'ROI'
enable_photoZ_interaction = True
initialize_photoZ = False

In [ ]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir,
                      pre_file=photoZ_preference_file)
    if initialize_photoZ:
        aPhz.prepare_photoZ()
pa.alert("Load your ROIs before continuing.")

In [ ]:
aDAT = AutoDAT()
created_file_map = {}
for subdir, dirs, files in os.walk(data_dir):
    
    # locate proper files in this subdir
    zda_file, roi_filepath, file_prefix, zda_filepath = None, None, None, None
    for file in files:
        if file.endswith('.zda'):
            zda_file = file
            zda_filepath = subdir + "/" + zda_file
            file_prefix = zda_file[:-4]

        # parse out the rec no. of the zda file
        rec_id = int(zda_file.split('_')[-1][:2])

        # look for a corresponding ROI file containing the keyword, r<rec_id>, and ending in .dat
        roi_filepath = None
        for f in files:
            if f.endswith('.dat') and roi_file_keyword in f and ('r' + str(rec_id)) in f:
                roi_filepath = subdir + "/" + f

        # filename validation
        if any([x is None for x in [zda_file, file_prefix, zda_filepath]]):
            print("File names not found or parsed properly. Check")
            print("zda_file, roi_filepath, file_prefix, zda_filepath")
            print(zda_file, roi_filepath, file_prefix, zda_filepath)
            continue
        
        # interact with photoZ
        if enable_photoZ_interaction:
            aPhz = AutoPhotoZ(data_dir=subdir)
            # open the PhotoZ file
            aPhz.select_PhotoZ()

            print("\n\nOpening", zda_file)
            aPhz.open_zda_file(zda_filepath)

            # Open the ROI file for this recording
            if roi_filepath is not None:
                print("Opening new ROI file", roi_filepath, "for", zda_file)
                aPhz.select_roi_tab()
                aPhz.open_roi_file(roi_filepath)
            else:
                print("No ROI file found for", zda_file, 'Continuing with last ROI file')

            # organized by subdir. Presumably you could replace this with date
            #  (e.g. parse the date out of your subdir name)
            created_file_map[subdir] = {}

            for i_trial in range(1, n_trials_per_recording + 1):

                aDAT.increment_trial()

                # save SNR values from PhotoZ
                snr_filename = subdir + "/" + file_prefix + "SNR.dat"
                if enable_photoZ_interaction:
                    aPhz.select_SNR_trace_value()
                    aPhz.save_trace_values(snr_filename)

                # save half-width values from PhotoZ
                hw_filename = subdir + "/" + file_prefix + "hw.dat" 
                if enable_photoZ_interaction:
                    aPhz.select_half_width_trace_value()
                    aPhz.save_trace_values(hw_filename)

                # save amp values from PhotoZ
                amp_filename = subdir + "/" + file_prefix + "amp.dat"
                if enable_photoZ_interaction:
                    aPhz.select_maxamp_trace_value()
                    aPhz.save_trace_values(amp_filename)

                # save latency values from PhotoZ
                latency_filename = subdir + "/" + file_prefix + "latency.dat"
                if enable_photoZ_interaction:
                    aPhz.select_latency_trace_value()
                    aPhz.save_trace_values(latency_filename)

                created_file_map[subdir][i_trial] = [
                    snr_filename,
                    hw_filename,
                    amp_filename,
                    latency_filename
                ]


In [ ]:
# read in the created files and store the data in a DataFrame
def read_dat_trace_file(filename):
    data_arr = pd.read_csv(filename,
                                 sep='\t',
                                 header=None,
                                 names=['ROI',  'Value'])
    return data_arr
    
file_data = []
for subdir in created_file_map:
    for i_trial in created_file_map:
        
        snr_fn, hw_fn, amp_fn, latency_fn = created_file_map[subdir][i_trial]

        amp = read_dat_trace_file(amp_fn)
        hw = read_dat_trace_file(hw_fn)
        df = read_dat_trace_file(snr_fn)
        lat = read_dat_trace_file(latency_fun)

        total_before_filter += len(amp['Value'])

        df = df.join(amp, rsuffix='_Amp')
        df = df.join(hw, rsuffix='_hw')
        df = df.join(lat, rsuffix='_latency')
        
        tr_col = [i_trial for _ in range(len(df['ROI']))]
        subdir_col = [subdir for _ in range(len(df['ROI']))]

        df.insert(4, 'Trial', tr_col)
        df.insert(5, 'Subdir/Date ID', subdir_col)

        file_data.append(df)

data_by_trial = pd.concat(file_data)

# Drop some unneeded columns
data_by_trial = data_by_trial.drop(columns=['Index_Amp',
                                            'Index_hw',
                                            'Index_latency'])
# rename some columns
data_by_trial = data_by_trial.rename(columns=
                                    {'Value': 'SNR',
                                     'Value_Amp': 'MaxAmp',
                                     'Value_hw': 'HalfWidth',
                                     'Value_latency': 'Latency'})

data_by_trial

In [ ]:
# save dataframe to a CSV file
csv_filename = "data_by_trial.csv"
df.to_csv(data_dir + csv_filename, index=False)
print("Wrote data to file", csv_filename)